I have a warning here, not sure how/if I need to deal with it
Looking at https://spatialreference.org/ref/epsg/3035/ they are the same?

In [1]:
!gdalinfo in_data/CLC/u2018_clc2018_v2020_20u1_raster100m/DATA/U2018_CLC2018_V2020_20u1.tif

Driver: GTiff/GeoTIFF
Files: in_data/CLC/u2018_clc2018_v2020_20u1_raster100m/DATA/U2018_CLC2018_V2020_20u1.tif
       in_data/CLC/u2018_clc2018_v2020_20u1_raster100m/DATA/U2018_CLC2018_V2020_20u1.tif.aux.xml
Size is 65000, 46000
Warning 1: The definition of projected CRS EPSG:3035 got from GeoTIFF keys is not the same as the one from the EPSG registry, which may cause issues during reprojection operations. Set GTIFF_SRS_SOURCE configuration option to EPSG to use official parameters (overriding the ones from GeoTIFF keys), or to GEOKEYS to use custom values from GeoTIFF keys and drop the EPSG code.
Coordinate System is:
PROJCRS["ETRS_1989_LAEA",
    BASEGEOGCRS["ETRS89",
        DATUM["European Terrestrial Reference System 1989",
            ELLIPSOID["GRS 1980",6378137,298.257222101004,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4258]],
    CONVERSION["Lambert Azimuthal Equal Area",
     

In [2]:
!gdalinfo -mm in_data/CLC/u2018_clc2018_v2020_20u1_raster100m/DATA/U2018_CLC2018_V2020_20u1.tif | grep Computed

Warning 1: The definition of projected CRS EPSG:3035 got from GeoTIFF keys is not the same as the one from the EPSG registry, which may cause issues during reprojection operations. Set GTIFF_SRS_SOURCE configuration option to EPSG to use official parameters (overriding the ones from GeoTIFF keys), or to GEOKEYS to use custom values from GeoTIFF keys and drop the EPSG code.
  Min=1.000 Max=48.000   Computed Min/Max=-128.000,44.000


In [2]:
!gdalinfo -hist in_data/CLC/u2018_clc2018_v2020_20u1_raster100m/DATA/U2018_CLC2018_V2020_20u1.tif

Driver: GTiff/GeoTIFF
Files: in_data/CLC/u2018_clc2018_v2020_20u1_raster100m/DATA/U2018_CLC2018_V2020_20u1.tif
       in_data/CLC/u2018_clc2018_v2020_20u1_raster100m/DATA/U2018_CLC2018_V2020_20u1.tif.aux.xml
Size is 65000, 46000
Warning 1: The definition of projected CRS EPSG:3035 got from GeoTIFF keys is not the same as the one from the EPSG registry, which may cause issues during reprojection operations. Set GTIFF_SRS_SOURCE configuration option to EPSG to use official parameters (overriding the ones from GeoTIFF keys), or to GEOKEYS to use custom values from GeoTIFF keys and drop the EPSG code.
Coordinate System is:
PROJCRS["ETRS_1989_LAEA",
    BASEGEOGCRS["ETRS89",
        DATUM["European Terrestrial Reference System 1989",
            ELLIPSOID["GRS 1980",6378137,298.257222101004,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4258]],
    CONVERSION["Lambert Azimuthal Equal Area",
     

In [104]:
!gdal_edit.py -a_nodata 48 in_data/CLC/u2018_clc2018_v2020_20u1_raster100m/DATA/U2018_CLC2018_V2020_20u1.tif

In [105]:
!gdalinfo in_data/CLC/u2018_clc2018_v2020_20u1_raster100m/DATA/U2018_CLC2018_V2020_20u1.tif | grep NoData

Warning 1: The definition of projected CRS EPSG:3035 got from GeoTIFF keys is not the same as the one from the EPSG registry, which may cause issues during reprojection operations. Set GTIFF_SRS_SOURCE configuration option to EPSG to use official parameters (overriding the ones from GeoTIFF keys), or to GEOKEYS to use custom values from GeoTIFF keys and drop the EPSG code.
  NoData Value=48


In the legend file the row counts are the actual values in the tif. 
Except for nodata which is 48, not 44

In [3]:
!paste <(seq 1 44) in_data/CLC/u2018_clc2018_v2020_20u1_raster100m/Legend/CLC2018_CLC2018_V2018_20_QGIS.txt | grep forest


reclassify to forest/non-forest
Lets make a recoding file first

In [6]:
%%bash
for((i=1;i<=44;++i));do
  if(($i >21 && $i < 26));then
    echo "$i 1"
  else
    echo "$i 0";fi
done > /tmp/recode4bash_CLC100.txt 

echo "-128 48" >> /tmp/recode4bash_CLC100.txt 
echo "128 48" >> /tmp/recode4bash_CLC100.txt 

In [5]:
!head -5 /tmp/recode4bash_CLC100.txt 
!echo -----------------------------
!grep "1$" /tmp/recode4bash_CLC100.txt 
!echo -----------------------------
!tail -5 /tmp/recode4bash_CLC100.txt 

1 0
2 0
3 0
4 0
5 0
-----------------------------
22 1
23 1
24 1
25 1
-----------------------------
42 0
43 0
44 0
-128 48
128 48


In [119]:
%%bash
start=`date +%s`
pkreclass -co COMPRESS=DEFLATE -co ZLEVEL=9                                          \
--code /tmp/recode4bash_CLC100.txt                                                   \
-nodata 48                                                                           \
-i in_data/CLC/u2018_clc2018_v2020_20u1_raster100m/DATA/U2018_CLC2018_V2020_20u1.tif \
-o out_data/CLC/CLC_recode01.tif                                                     \

end=`date +%s`

expr $end - $start

0...10...20...30...40...50...60...70...80...90...100 - done.
225


Warning 1: The definition of projected CRS EPSG:3035 got from GeoTIFF keys is not the same as the one from the EPSG registry, which may cause issues during reprojection operations. Set GTIFF_SRS_SOURCE configuration option to EPSG to use official parameters (overriding the ones from GeoTIFF keys), or to GEOKEYS to use custom values from GeoTIFF keys and drop the EPSG code.


In [8]:
!gdalinfo out_data/CLC/CLC_recode01.tif 

Driver: GTiff/GeoTIFF
Files: out_data/CLC/CLC_recode01.tif
Size is 65000, 46000
Coordinate System is:
PROJCRS["ETRS89-extended / LAEA Europe",
    BASEGEOGCRS["ETRS89",
        ENSEMBLE["European Terrestrial Reference System 1989 ensemble",
            MEMBER["European Terrestrial Reference Frame 1989"],
            MEMBER["European Terrestrial Reference Frame 1990"],
            MEMBER["European Terrestrial Reference Frame 1991"],
            MEMBER["European Terrestrial Reference Frame 1992"],
            MEMBER["European Terrestrial Reference Frame 1993"],
            MEMBER["European Terrestrial Reference Frame 1994"],
            MEMBER["European Terrestrial Reference Frame 1996"],
            MEMBER["European Terrestrial Reference Frame 1997"],
            MEMBER["European Terrestrial Reference Frame 2000"],
            MEMBER["European Terrestrial Reference Frame 2005"],
            MEMBER["European Terrestrial Reference Frame 2014"],
            ELLIPSOID["GRS 1980",6378137,298

In [7]:
!gdalinfo -mm in_data/CLC/u2018_clc2018_v2020_20u1_raster100m/DATA/U2018_CLC2018_V2020_20u1.tif | grep Computed
!echo ---------------------------------
!gdalinfo -mm out_data/CLC/CLC_recode01.tif  | grep Computed

Warning 1: The definition of projected CRS EPSG:3035 got from GeoTIFF keys is not the same as the one from the EPSG registry, which may cause issues during reprojection operations. Set GTIFF_SRS_SOURCE configuration option to EPSG to use official parameters (overriding the ones from GeoTIFF keys), or to GEOKEYS to use custom values from GeoTIFF keys and drop the EPSG code.
  Min=1.000 Max=48.000   Computed Min/Max=-128.000,44.000
---------------------------------
    Computed Min/Max=0.000,1.000


In [9]:
!gdalinfo -hist in_data/CLC/u2018_clc2018_v2020_20u1_raster100m/DATA/U2018_CLC2018_V2020_20u1.tif

Driver: GTiff/GeoTIFF
Files: in_data/CLC/u2018_clc2018_v2020_20u1_raster100m/DATA/U2018_CLC2018_V2020_20u1.tif
       in_data/CLC/u2018_clc2018_v2020_20u1_raster100m/DATA/U2018_CLC2018_V2020_20u1.tif.aux.xml
Size is 65000, 46000
Warning 1: The definition of projected CRS EPSG:3035 got from GeoTIFF keys is not the same as the one from the EPSG registry, which may cause issues during reprojection operations. Set GTIFF_SRS_SOURCE configuration option to EPSG to use official parameters (overriding the ones from GeoTIFF keys), or to GEOKEYS to use custom values from GeoTIFF keys and drop the EPSG code.
Coordinate System is:
PROJCRS["ETRS_1989_LAEA",
    BASEGEOGCRS["ETRS89",
        DATUM["European Terrestrial Reference System 1989",
            ELLIPSOID["GRS 1980",6378137,298.257222101004,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4258]],
    CONVERSION["Lambert Azimuthal Equal Area",
     

In [77]:
!gdalinfo     out_data/CLC/CLC_recode01.tif  | grep NoData

  NoData Value=48
